In [1]:
import ray
from time import time 
from matplotlib import pyplot as plt
from numpy.linalg import norm
from Quantum_functions import *
import numpy as np
from QuantumPFunctions import *

ModuleNotFoundError: No module named 'ray'

In [2]:
gb = 1024*1024*1024

from qutip import *
# Start Ray.
ray.init()
#ray.init(memory =14*gb, object_store_memory = 12*gb)

2020-04-01 00:57:57,487	WARNING services.py:586 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-04-01 00:57:57,488	INFO resource_spec.py:212 -- Starting Ray with 7.52 GiB memory available for workers and up to 3.76 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-04-01 00:57:57,827	INFO services.py:1078 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.1.97',
 'redis_address': '192.168.1.97:28212',
 'object_store_address': '/tmp/ray/session_2020-04-01_00-57-57_486876_8232/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-04-01_00-57-57_486876_8232/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-04-01_00-57-57_486876_8232'}

In [3]:
n = 8
processors = 4
state1 = tensor([basis(2,0)]*n)

alpha1 = Heisenberg1dRingGen(1,1,1,n)
alpha0 = lambda n,m,i,j: 0
beta1 = lambda n,i : [0,0,1][i]
beta0 = lambda n,i :0

self_interaction = hamiltonian(alpha1,beta0,n)
self_interaction = self_interaction/self_interaction.norm()
external_interaction = hamiltonian(alpha0,beta1,n)
external_interaction = external_interaction/external_interaction.norm()


perturbation: Qobj = make_hermitian(rand_unitary(2**n, dims = self_interaction.dims))
perturbation = perturbation/perturbation.norm()
epsilon = 0.3

H1 = epsilon*self_interaction + external_interaction + epsilon**2*perturbation
H2 = external_interaction
H3 = external_interaction + epsilon*perturbation

### Testing Trace vs Energy Compare

In [ ]:
start = time()
xs,ys = energy_trace_comp_2d(H1,1,1)
end = time()

single_t = end-start

plt.scatter(xs,ys,s=1)
plt.show()
print(f"Finished executing completed in {end-start}s")

In [ ]:
start = time()
xs,ys = energy_trace_compare_p(H1,1,1,proc =processors)
end = time()

mulit_t  = end-start
print(len(xs))

plt.scatter(xs,ys,s=1)
plt.show()

print(f"Finished executing completed in {end-start}s")


In [ ]:
print(f"System obtains a speedup of {single_t/mulit_t} using {processors} processors" )

### Testing simulation compare

In [ ]:
n=6
state1 = tensor([basis(2,0)]*n)
processors = 4
beta0 = lambda n,m: 0
alpha1 = Heisenberg1dRingGen(-1,1,1,n)

H1 = hamiltonian(alpha1,beta0, n)
H1=H1/H1.norm()

assert(H1.isherm)

In [ ]:
start = time()
equilibration_analyser(H1,state1,1e5,200)
end=time()
print(f"Finished executing completed in {end-start}s")
sim_s = end-start

In [ ]:
start = time()
energys,states = H1.eigenstates()
print("Found states")
equilibration_analyser_p(energys,states,state1,0,1e5,200,Proc=4)
end=time()
print(f"Finished executing completed in {end-start}s")
sim_m = end-start

In [ ]:
print(f"System obtains a speedup of {sim_s/sim_m} using {processors} processors" )

In [ ]:
energys,states = H1.eigenstates()

In [ ]:
print(states[0])
print(ket2dm(states[0]))



In [3]:
n = 3

initial_state = tensor([basis(2,0)]*n)

alpha1 = Heisenberg1dRingGen(1,1,1,n)
alpha0 = lambda n,m,i,j: 0
beta1 = lambda n,i : [0,0,1][i]
beta0 = lambda n,i :0

self_interaction = hamiltonian(alpha1,beta0,n)
self_interaction = self_interaction/self_interaction.norm()
external_interaction = hamiltonian(alpha0,beta1,n)
external_interaction = external_interaction/external_interaction.norm()


perturbation: Qobj = make_hermitian(rand_unitary(2**n, dims = self_interaction.dims))
perturbation = perturbation/perturbation.norm()
epsilon = 0.3

H1 = epsilon*self_interaction + external_interaction + epsilon**2*perturbation
H2 = external_interaction
H3 = external_interaction + epsilon*perturbation
H4 = external_interaction + epsilon*self_interaction

In [4]:
energys,states= H3.eigenstates()

In [5]:
coef = [initial_state.overlap(estate) for estate in states]

In [9]:

eq_state = get_equilibrated_dens_op_P(energys,states,coef,proc=4)
eq_old = get_equilibrated_dens_op(H3,initial_state)



3


In [7]:
print(eq_old)

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[ 9.61663713e-01+0.00000000e+00j -3.45153685e-02-1.75585836e-02j
   4.77048811e-02-2.78020414e-02j -2.96754293e-02-1.35932568e-02j
  -4.32018816e-02-6.50903880e-02j -2.55537658e-02-6.15025540e-02j
   1.61740709e-02+2.32313443e-02j -3.05878068e-02-1.57136435e-02j]
 [-3.45153685e-02+1.75585836e-02j  3.79868027e-03+0.00000000e+00j
  -1.87301392e-03+1.36021062e-03j  1.22426940e-03-5.95786851e-05j
   3.47859601e-03+2.57813355e-03j  2.22228803e-03+1.64361671e-03j
  -8.28634351e-04-4.48842410e-04j  1.35693446e-03+1.22599098e-06j]
 [ 4.77048811e-02+2.78020414e-02j -1.87301392e-03-1.36021062e-03j
   6.38764797e-03+0.00000000e+00j -1.12647583e-03-1.52986154e-03j
   4.85058682e-04-4.05019884e-03j  7.10016208e-04-3.89938017e-03j
   2.44618259e-04+1.53114159e-03j -9.99881742e-04-1.55942653e-03j]
 [-2.96754293e-02+1.35932568e-02j  1.22426940e-03+5.95786851e-05j
  -1.12647583e-03+1.52986154e-03j  2.

In [8]:
print(eq_state)

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[ 4.26209819e-03+0.00000000e+00j -1.20232975e-04+3.97112849e-05j
   1.13405107e-04-3.67952419e-05j  2.73228659e-05+1.17624730e-06j
  -8.67272302e-05-1.95570970e-04j -6.79607285e-05-8.23870799e-05j
  -6.50681547e-05-8.58151303e-06j -8.68053091e-05-3.30009547e-05j]
 [-1.20232975e-04-3.97112849e-05j  3.34453819e-03+0.00000000e+00j
   9.12410154e-04+6.34768934e-04j  5.99070480e-05-4.54021648e-05j
  -3.83047043e-04-6.03360882e-04j -1.18922328e-04-2.61091370e-05j
   4.07073541e-05+6.84706266e-05j  1.69004611e-04+9.41880141e-06j]
 [ 1.13405107e-04+3.67952419e-05j  9.12410154e-04-6.34768934e-04j
   2.12174267e-03+0.00000000e+00j  2.41874921e-05+3.15104181e-05j
  -3.82979810e-04-1.83802621e-04j  2.16516846e-05+1.43336726e-04j
  -1.53975906e-04+5.67172764e-05j  7.46344427e-05-5.41583904e-05j]
 [ 2.73228659e-05-1.17624730e-06j  5.99070480e-05+4.54021648e-05j
   2.41874921e-05-3.15104181e-05j  4.